In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

In [6]:
mnist_data,mnist_info = tfds.load(name = 'mnist',with_info = True,as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/1 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Extraction completed...:   0%|          | 0/1 [00:00<?, ? file/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0

In [7]:
mnist_data

{'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [9]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [23]:
mnist_train, mnist_test = mnist_data['train'],mnist_data['test']
# define a validation set
num_mnist_validation = 0.1*mnist_info.splits['train'].num_examples
num_mnist_validation = tf.cast(mnist_validation,tf.int64)
num_mnist_test_sample = mnist_info.splits['test'].num_examples
num_mnist_test_sample = tf.cast(mnist_test_sample,tf.int64)
# defining a function to scale the intensity of the images from 0 to 1
def scale(image,label):
    image   =  tf.cast(image,tf.float32)
    image = image/255.
    return image, label
scaled_train_validation_data = mnist_train.map(scale) # scaling 
scaled_test_data = mnist_test.map(scale) # scaling

## shuffling and batching the data 

In [29]:
BUFFER_SIZE = 10000
shuffled_validation_train_data = scaled_train_validation_data.shuffle(BUFFER_SIZE) # shuffle the whole data
shuffled_validation_data = shuffled_validation_train_data.take(num_mnist_validation)
shuffled_train_data = shuffled_validation_train_data.skip(num_mnist_validation)

#batching
BATCH_SIZE = 100
shuffled_train_data = shuffled_train_data.batch(BATCH_SIZE)
  #FOR Validation test data we need to specify the whole batch as one batch since we need to move forward and validation  do not have 
  #back propagation in or we dint use it in calculating  weight but still we should define a batch for validation data too
shuffled_validation_data = shuffled_validation_data.batch(num_mnist_validation)
test_data  = scaled_test_data.batch(num_mnist_test_sample)

# extract validation inputs and targets
validation_inputs, validation_targets = next(iter(shuffled_validation_data))






## MODEL
 #outline the model

In [33]:
input_size = 784
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([ 
            tf.keras.layers.Flatten(input_shape = (28,28,1)),
            tf.keras.layers.Dense(hidden_layer_size,activation ='relu'),
            tf.keras.layers.Dense(hidden_layer_size,activation ='relu'),
            tf.keras.layers.Dense(output_size,activation='softmax')





])

## choose the loss and the optimizer

In [35]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

## training

In [37]:
model.fit(shuffled_train_data,epochs = 5, validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 9s - loss: 0.4252 - accuracy: 0.8778 - val_loss: 0.2101 - val_accuracy: 0.9385
Epoch 2/5
540/540 - 3s - loss: 0.1902 - accuracy: 0.9448 - val_loss: 0.1510 - val_accuracy: 0.9558
Epoch 3/5
540/540 - 3s - loss: 0.1411 - accuracy: 0.9587 - val_loss: 0.1316 - val_accuracy: 0.9605
Epoch 4/5
540/540 - 3s - loss: 0.1153 - accuracy: 0.9657 - val_loss: 0.1221 - val_accuracy: 0.9667
Epoch 5/5
540/540 - 3s - loss: 0.0969 - accuracy: 0.9709 - val_loss: 0.0893 - val_accuracy: 0.9743


In [39]:
test_loss,test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 4ms/step - loss: 0.1045 - accuracy: 0.9686


In [41]:
print(' test loss {0:.2f}. test accuracy{1:.2f}'.format(test_loss,test_accuracy*100))

test loss 0.10. test accuracy96.86
